In [4]:
import torch
import tqdm
from sentence_transformers import SentenceTransformer

class BGEEmbedder:
    def __init__(self, model_name="BAAI/bge-m3"):
        self.model = SentenceTransformer(model_name)
        self.prefix = "Represent this sentence for searching relevant passages: "

    def embed(self, texts, batch_size=64):
        # BGE-M3 yêu cầu prefix cho truy vấn và văn bản
        texts_with_prefix = [
            text if text.startswith(self.prefix) else self.prefix + text
            for text in texts
        ]

        # Dùng encode với batch size và normalize sẵn
        embeddings = self.model.encode(
            texts_with_prefix,
            batch_size=batch_size,
            normalize_embeddings=True,
            show_progress_bar=True
        )
        return embeddings

In [19]:
import pandas as pd

course_summaries = pd.read_csv(r'D:\Learn\Semester_5\SEG301\Fap-Chat\data\FAP\FINAL\course_summaries.csv')
grade_details = pd.read_csv(r'D:\Learn\Semester_5\SEG301\Fap-Chat\data\FAP\FINAL\grade_details.csv')
student_profile = pd.read_csv(r'D:\Learn\Semester_5\SEG301\Fap-Chat\data\FAP\student_profile.csv')
attendance_reports = pd.read_csv(r'D:\Learn\Semester_5\SEG301\Fap-Chat\data\FAP\FINAL\attendance_reports.csv')


In [17]:
print(course_summaries.columns)
print(grade_details.columns)
print(student_profile.columns)
print(attendance_reports.columns)

Index(['term', 'course_name', 'course_code', 'avg_score', 'status', 'summary'], dtype='object')
Index(['student_id', 'term', 'course_name', 'course_code', 'category', 'item',
       'weight', 'value'],
      dtype='object')
Index(['full_name', 'date_of_birth', 'gender', 'id_card_number',
       'home_address', 'phone_number', 'email_address', 'id_date_of_issue',
       'id_place_of_issue', 'parent_full_name', 'parent_phone_number',
       'parent_address', 'parent_email', 'parent_job', 'parent_workplace',
       'roll_number', 'old_roll_number', 'member_code', 'enrollment_date',
       'study_mode', 'current_status', 'current_term_number', 'major',
       'curriculum', 'capstone_project', 'main_class', 'specialization',
       'account_balance', 'previous_major', 'decision_graduate_check',
       'is_full_time_student', 'full_time_confirmed_date',
       'is_scholarship_student', 'valid_study_period', 'training_type',
       'decision_dropout', 'decision_transfer_campus',
       'decis

In [4]:
# Tạo các json cho FAP
import json
coursesummary_json_chunks = course_summaries.to_dict(orient='records')
print(coursesummary_json_chunks[0])
attendancerp_json_chunks = attendance_reports.to_dict(orient='records')
print(attendancerp_json_chunks[0])
gradedetail_json_chunks = grade_details.to_dict(orient='records')
print(gradedetail_json_chunks[0])
studentprofile_json_chunks = student_profile.to_dict(orient='records')
print(studentprofile_json_chunks[0])

{'term': 'Fall2024', 'course_name': 'AI, DS with Python & SQL ', 'course_code': 'ADY201m', 'avg_score': 8.6, 'status': 'Passed', 'summary': "{'Assignment': {'weight': '10.0 %', 'value': '9'}, 'Progress test 1': {'weight': '10.0 %', 'value': '9.4'}, 'Progress test 2': {'weight': '10.0 %', 'value': '9'}, 'Lab': {'weight': '30.0 %', 'value': '9.3'}, 'Final exam': {'weight': '40.0 %', 'value': '7.6'}, 'Final exam Resit': {'weight': '40.0 %', 'value': None}}"}
{'student_id': 'DE190340', 'term': 'Fall2024', 'course_name': 'AI, DS with Python & SQL', 'course_code': 'ADY201m', 'no': 2, 'date': 'Monday 09/09/2024', 'slot': '3_(12:30-14:45)', 'room': 'Gamma 410.', 'lecturer': 'VuBC', 'group': 'AI18D02', 'status': 'Present', 'comment': 'unknown'}
{'student_id': 'DE190340', 'term': 'Fall2024', 'course_name': 'AI, DS with Python & SQL ', 'course_code': 'ADY201m', 'category': 'Assignment', 'item': 'Assignment', 'weight': '10.0 %', 'value': '9.0'}
{'full_name': 'Phan Công Tâm', 'date_of_birth': '19/0

In [195]:
import os
out_dir = r'D:\Desktop\tap tanh hoc code\.vscode\Summer_2025\SEG301\Fap-Chat\data\FAP\FINAL\FAP_Chunk_JSON'
os.makedirs(out_dir, exist_ok=True)

# Mapping tên file → biến
chunk_map = {
    "coursesummary_json_chunks.json": coursesummary_json_chunks,
    "attendancerp_json_chunks.json": attendancerp_json_chunks,
    "gradedetail_json_chunks.json": gradedetail_json_chunks,
    "studentprofile_json_chunks.json": studentprofile_json_chunks,
}

# Ghi lần lượt
for filename, data in chunk_map.items():
    filepath = os.path.join(out_dir, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [46]:
def build_attendance_payload(row):
    def safe(x):
        return str(x).strip() if pd.notnull(x) else "Unknown"
    
    return {
        "student_id": safe(row["student_id"]),
        "semsester": safe(row["term"]),
        "subject_name": safe(row["course_name"]),
        "subject_code": safe(row["course_code"]),
        "session_number": int(row["no"]) if pd.notnull(row["no"]) else -1,
        "date": safe(row["date"]),
        "time_slot": safe(row["slot"]),
        "room": safe(row["room"]),
        "lecturer": safe(row["lecturer"]),
        "group": safe(row["group"]),
        "status": safe(row["status"]),  # e.g., "present", "absent"
        "comment": safe(row["comment"]),
        "type": "attendance",
        "content": (
            f"TYPE: Attendance\n"
            f"Student: {safe(row['student_id'])}\n"
            f"Course: {safe(row['course_code'])} - {safe(row['course_name'])}\n"
            f"Semeseter: {safe(row['term'])} | Session No.: {safe(row['no'])} - Date: {safe(row['date'])} - Slot: {safe(row['slot'])} - Room: {safe(row['room'])}\n"
            f"Lecturer: {safe(row['lecturer'])} | Group: {safe(row['group'])}\n"
            f"Status: {safe(row['status'])} | Comment: {safe(row['comment'])}"
        )
    }

attendance_payloads = [build_attendance_payload(row) for _, row in attendance_reports.iterrows()]
print(len(attendance_payloads))


538


In [47]:
def build_course_summary_payload(row):
    def safe(x):
        return str(x).strip() if pd.notnull(x) else "Unknown"

    return {
        "student_id": 'DE190340',
        "semester": safe(row["term"]),
        "subject_name": safe(row["course_name"]),
        "subject_code": safe(row["course_code"]),
        "average_score": float(row["avg_score"]) if pd.notnull(row["avg_score"]) else -1.0,
        "status": safe(row["status"]),
        "summary": safe(row["summary"]),
        "type": "course summary",
        "content": (
            f"TYPE: Course Summary\n"
            f"Course: {safe(row['course_code'])} - {safe(row['course_name'])}\n"
            f"Term: {safe(row['term'])}\n"
            f"Average Score: {safe(row['avg_score'])}\n"
            f"Status: {safe(row['status'])}\n"
            f"Summary: {safe(row['summary'])}"
        )
    }

co_sum_payloads = [build_course_summary_payload(row) for _, row in course_summaries.iterrows()]
print(len(co_sum_payloads))


27


In [48]:
def build_grade_detail_payload(row):
    def safe(x):
        return str(x).strip() if pd.notnull(x) else "Unknown"

    return {
        "student_id": safe(row["student_id"]),
        "semester": safe(row["term"]),
        "subject_name": safe(row["course_name"]),
        "subject_code": safe(row["course_code"]),
        "category": safe(row["category"]),
        "assessment_item": safe(row["item"]),
        "weight": safe(row["weight"]),
        "score_value": row["value"] if pd.notnull(row["value"]) else -1.0,
        "type": "grade detail",
        "content": (
            f"TYPE: Grade Detail\n"
            f"Student: {safe(row['student_id'])}\n"
            f"Course: {safe(row['course_code'])} - {safe(row['course_name'])}\n"
            f"Semester: {safe(row['term'])}\n"
            f"Item: {safe(row['item'])} | Category: {safe(row['category'])}\n"
            f"Weight: {safe(row['weight'])} | Score: {safe(row['value'])}"
        )
    }

grade_detail_payloads = [build_grade_detail_payload(row) for _, row in grade_details.iterrows()]
print(len(grade_detail_payloads))


167


In [31]:
def build_student_profile_payload(row):
    def safe(x):
        return str(x).strip() if pd.notnull(x) else "Unknown"

    return {
        "full_name": safe(row["full_name"]),
        "date_of_birth": safe(row["date_of_birth"]),
        "gender": safe(row["gender"]),
        "id_card_number": safe(row["id_card_number"]),
        "id_date_of_issue": safe(row["id_date_of_issue"]),
        "id_place_of_issue": safe(row["id_place_of_issue"]),
        "home_address": safe(row["home_address"]),
        "phone_number": safe(row["phone_number"]),
        "email_address": safe(row["email_address"]),
        "roll_number": safe(row["roll_number"]),
        "old_roll_number": safe(row["old_roll_number"]),
        "member_code": safe(row["member_code"]),
        "enrollment_date": safe(row["enrollment_date"]),
        "major": safe(row["major"]),
        "main_class": safe(row["main_class"]),
        "current_status": safe(row["current_status"]),
        "is_full_time_student": bool(row["is_full_time_student"]),
        "is_scholarship_student": bool(row["is_scholarship_student"]),
        "training_type": safe(row["training_type"]),
        "start_term": safe(row["start_term"]),
        "type": "student profile",
        "content": (
            f"Student Profile: {safe(row['full_name'])} | ID: {safe(row['roll_number'])}\n"
            f"Date of Birth: {safe(row['date_of_birth'])} | Gender: {safe(row['gender'])}\n"
            f"Major: {safe(row['major'])} | Class: {safe(row['main_class'])} | Status: {safe(row['current_status'])}\n"
            f"Training Type: {'Full-time' if bool(row['is_full_time_student']) else 'Part-time'} | Scholarship: {'Yes' if bool(row['is_scholarship_student']) else 'No'}\n"
            f"Address: {safe(row['home_address'])} | Email: {safe(row['email_address'])} | Phone: {safe(row['phone_number'])}"
        )
    }


profile_payload = [build_student_profile_payload(row) for _, row in student_profile.iterrows()]
print(len(profile_payload))


1


In [49]:
payloads_total = profile_payload + grade_detail_payloads + co_sum_payloads + attendance_payloads
print(len(payloads_total), "total payloads created.")


733 total payloads created.


In [5]:
# Khởi tạo embedder
embedder = BGEEmbedder()

c:\Users\DO TUAN MINH\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\DO TUAN MINH\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [1]:
from deep_translator import GoogleTranslator

def translate_vi_to_en_google(text):
    return GoogleTranslator(source='vi', target='en').translate(text)

In [12]:


from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Load mô hình dịch
model_id = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Pipeline dịch offline
translator = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    src_lang="vie",
    tgt_lang="eng_Latn",
    max_length=512
)

def translate_vi_to_en_local(text):
    result = translator(text)
    return result[0]['translation_text']



c:\Users\DO TUAN MINH\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("vinai/vinai-translate-vi2en")
model = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-vi2en")

def translate_vi2en(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)


c:\Users\DO TUAN MINH\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

c:\Users\DO TUAN MINH\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DO TUAN MINH\.cache\huggingface\hub\models--vinai--vinai-translate-vi2en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sentencepiece.bpe.model:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.79G [00:00<?, ?B/s]

In [23]:
query = ["điểm danh học kì này"]  # [VI] truy vấn
query=[translate_vi_to_en_google(query[0])]
print(query)



['Attendance this semester']


In [24]:
texts = [
    "The student was marked present in today's attendance record.",    # [EN] đúng
    "The attendance list was updated by the teacher today.",           # [EN] đúng
    "The final exam score was released this morning.",                 # [EN] sai
    "Grades for the midterm have been posted on the portal.",          # [EN] sai
]

query = ["điểm danh môn CPV học kì trước"]  # [VI] truy vấn
query=[translate_vi_to_en_google(query[0])]
print(query)
# Embed
all_texts = query + texts
embeddings = embedder.embed(all_texts)

# Tính cosine similarity giữa query và các câu trong database
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

query_emb = embeddings[0]
print("Similarity between query and database entries:\n")
for i, text in enumerate(texts):
    sim = cosine_similarity(query_emb, embeddings[i + 1])
    print(f"{sim:.4f} : {text}")


['Attendance CPV for the previous semester']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity between query and database entries:

0.7197 : The student was marked present in today's attendance record.
0.7013 : The attendance list was updated by the teacher today.
0.6300 : The final exam score was released this morning.
0.6313 : Grades for the midterm have been posted on the portal.


In [ ]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

sim = cosine_similarity(embeddings[0], embeddings[1])
print(f"Cosine similarity: {sim}")


In [50]:
# Only include payloads that have the 'noi_dung' key
contents = [payload["content"] for payload in payloads_total if "content" in payload]
embedder = BGEEmbedder()
embeddings = embedder.embed(contents, batch_size=64)

print(len(embeddings), "embeddings generated.")

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

733 embeddings generated.


In [34]:
print("Embedding shape:", len(contents))

Embedding shape: 733


In [35]:
print(embeddings[0].shape)

(1024,)


In [36]:
from collections import Counter

types = [p["type"] for p in payloads_total if "type" in p]
print(Counter(types))

Counter({'attendance': 538, 'grade detail': 167, 'course summary': 27, 'student profile': 1})


In [51]:
from qdrant_client.models import PointStruct 
import uuid

points = [
    PointStruct(
        id=str(uuid.uuid4()),
        vector=vec.tolist(),
        payload=payload
    )
    for vec, payload in zip(embeddings, payloads_total)
]
print(points[0])

id='e5a19869-b441-4c2a-aa7d-e27ec2b36ee3' vector=[-0.021864915266633034, -0.018371619284152985, -0.03197566419839859, -0.04762747511267662, -0.03575019910931587, -0.00931945163756609, 0.02271103486418724, -0.04164685681462288, 0.022018978372216225, 0.04592790827155113, 0.00313775148242712, 0.007306504994630814, -0.033743780106306076, 0.010839799419045448, 0.022249720990657806, 0.0006440352881327271, -0.049894098192453384, -0.03302336111664772, 0.021032430231571198, -0.008073359727859497, -0.030714722350239754, 0.03631072863936424, 0.030450383201241493, 0.003917662426829338, 0.07265090197324753, -0.01197606697678566, 0.011288180947303772, -9.774034697329625e-05, -0.0053037917241454124, -0.01688930206000805, 0.0020776866003870964, 0.0021529917139559984, 0.08457872271537781, -0.04586654156446457, -0.0071980091743171215, -0.03617727383971214, 0.022130301222205162, -0.022903095930814743, -0.0648997351527214, 0.022557087242603302, 0.013195006176829338, -0.023907091468572617, -0.0268896762281

In [206]:
len(points)

733

# Clean vector data base

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
qdrant_api_key = os.getenv("qdrant_api_key")


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from qdrant_client.models import Filter
client = QdrantClient(
    url=r"https://0f47d391-b7c1-45d9-a956-5f7228cd80f3.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key=qdrant_api_key,
    prefer_grpc=False
)
print(client.get_collections())
# Xóa các collection cũ nếu có
# client.delete_collection("my_rag_vectors")
client.delete_collection("FINAL")

# print(client.get_collections())



collections=[CollectionDescription(name='Fap_data_testing'), CollectionDescription(name='flm_fap'), CollectionDescription(name='star_charts'), CollectionDescription(name='FINAL')]


True

In [53]:
client.recreate_collection(
    collection_name="FINAL",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
)
import time

def safe_upsert(client, collection_name, points, retries=5, wait=5):
    for attempt in range(retries):
        try:
            client.upsert(collection_name=collection_name, points=points)
            return True
        except Exception as e:
            print(f"❌ Retry {attempt+1}/{retries} - Lỗi: {e}")
            time.sleep(wait)
    return False
for i in range(0, len(points), 100):
    success = safe_upsert(client, "FINAL", points[i:i+100])
    if not success:
        print("⚠️ Không thể upload batch", i)


C:\Users\DO TUAN MINH\AppData\Local\Temp\ipykernel_24344\2711709356.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [42]:
count = client.count(collection_name="FINAL", exact=True).count
print(f"Total points in collection: {count}")

Total points in collection: 733


In [54]:
client.create_payload_index(
    collection_name="FINAL",
    field_name="type",
    field_schema="keyword"
)
client.create_payload_index(
    collection_name="FINAL",
    field_name="semester",  # Use ASCII-only field name
    field_schema="keyword"
)
client.create_payload_index(
    collection_name="FINAL",
    field_name="subject_code",
    field_schema="keyword"
)


UpdateResult(operation_id=13, status=<UpdateStatus.COMPLETED: 'completed'>)

In [57]:
scroll_offset = None
batch_size = 100

while True:
    points, scroll_offset = client.scroll(
        collection_name='flm_fap',
        limit=batch_size,
        offset=scroll_offset,
        with_payload=True,
        with_vectors=True
    )
    if not points:
        break

    print(f"✅ Copied {len(points)} points, current offset: {scroll_offset}")

    batch = [
        PointStruct(id=p.id, vector=p.vector, payload=p.payload)
        for p in points
    ]

    success = safe_upsert(client, "FINAL", batch)
    if not success:
        print("❌ Failed to upload batch at offset:", scroll_offset)

    


✅ Copied 100 points, current offset: 046e6e58-ad35-494f-9e51-c62838e2ab9f
✅ Copied 100 points, current offset: 08c4fbd3-22b0-4178-80cc-031f2f8f1b6d
✅ Copied 100 points, current offset: 0cb1d358-7390-4424-bb6c-a49329b7fc73
✅ Copied 100 points, current offset: 105b33ba-b3e7-4a56-9396-64f737837adb
✅ Copied 100 points, current offset: 150c00c9-cb9b-4f07-8cf8-23d685bb7842
✅ Copied 100 points, current offset: 1a46d564-d543-44ad-a46f-ad641ee4b9fc
✅ Copied 100 points, current offset: 1f33929f-61e4-45f5-9f28-8d8474d0a5ef
✅ Copied 100 points, current offset: 235afc33-1336-4534-bc76-63cdcb777303
✅ Copied 100 points, current offset: 286746d7-2ec7-4ed6-8085-83cd2ef80636
✅ Copied 100 points, current offset: 2d1bbc92-df55-4abb-840d-b751a838d423
✅ Copied 100 points, current offset: 31302850-6017-4c02-b107-66edaaa482dc
✅ Copied 100 points, current offset: 355b1dcb-c2d3-4e7d-baf4-ac73a1673c60
✅ Copied 100 points, current offset: 3a206550-d404-430f-89e3-b0c2e82fd768
✅ Copied 100 points, current offset: 3

KeyboardInterrupt: 

In [59]:
all_types = set()
scroll_offset = None
total_points = 0
batch_count = 0

while True:
    points, scroll_offset = client.scroll(
        collection_name="FINAL",
        limit=1000,
        offset=scroll_offset,
        with_payload=True,
        with_vectors=False
    )
    
    if not points:
        break

    for p in points:
        # Nếu payload có trường "type" thì thêm vào tập hợp
        if "type" in p.payload:
            all_types.add(p.payload["type"])

    total_points += len(points)
    batch_count += 1
    print(f"✅ Batch {batch_count} | Total scanned: {total_points} | Unique types so far: {len(all_types)}")

print("\n📌 Unique `type` values found in collection:")
for t in sorted(all_types):
    print("  -", t)

✅ Batch 1 | Total scanned: 1000 | Unique types so far: 11
✅ Batch 2 | Total scanned: 2000 | Unique types so far: 11
✅ Batch 3 | Total scanned: 3000 | Unique types so far: 11
✅ Batch 4 | Total scanned: 4000 | Unique types so far: 11
✅ Batch 5 | Total scanned: 5000 | Unique types so far: 11
✅ Batch 6 | Total scanned: 6000 | Unique types so far: 12
✅ Batch 7 | Total scanned: 6611 | Unique types so far: 12
✅ Batch 8 | Total scanned: 7611 | Unique types so far: 12


KeyboardInterrupt: 

In [60]:
for t in sorted(all_types):
    print("  -", t)

  - assessment
  - attendance
  - construtive_question
  - course summary
  - grade detail
  - guide
  - learning outcome
  - material
  - overview
  - session
  - student profile
  - student_list


In [211]:
# subject_alias = {
#     "OTP101": "Chương trình định hướng, Nghĩa vụ quân sự",
#     "VOV114": "Vovinam căn bản",
#     "DSA103": "Nhạc cụ truyền thống",
#     "CSI105": "Nhập môn Khoa học Máy tính",
#     "MAE101": "Toán cho kỹ thuật/Đại số tuyến tính",
#     "VOV124": "Vovinam nâng cao",
#     "MAD101": "Toán rời rạc",
#     "PFP191": "Lập trình Python cơ bản",
#     "SSL101c": "Kỹ năng học đại học",
#     "CEA201": "Kiến trúc máy tính",
#     "AIG202c": "Trí tuệ nhân tạo",
#     "VOV134": "Vovinam nâng cao 2",
#     "SSG104": "Kỹ năng giao tiếp và làm việc nhóm",
#     "DBI202": "Hệ quản trị cơ sở dữ liệu",
#     "CSD203": "Cấu trúc dữ liệu và giải thuật Python",
#     "ITE303c": "Đạo đức nghề nghiệp CNTT",
#     "MAI391": "Toán cao cấp",
#     "MAS291": "Xác suất thống kê",
#     "ADY201m": "AI, DS với Python & SQL",
#     "JPD113": "Tiếng Nhật sơ cấp 1-A1.1",
#     "JPD123": "Tiếng Nhật sơ cấp 1-A1.2",
#     "AIL303m": "Machine Learning",
#     "CPV301": "Thị giác máy tính",
#     "DAP391m": "Dự án AI-DS",
#     "SWE201c": "Nhập môn Kỹ thuật phần mềm",
#     "TMG301": "Text Mining",
#     "SEG301": "Search Engines",
#     "DPL302m": "Deep Learning",
#     "DWP301c": "Lập trình web với Python"
# }
subject_map = {row["course_code"]: f"{row["course_code"]} - {row["course_name"]}" 
               for _, row in attendance_reports[["course_code", "course_name"]].dropna().drop_duplicates().iterrows()}
subject_embeddings = {
    code: embedder.embed([embedder.prefix + name])[0]
    for code, name in subject_map.items()
}

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [212]:
# print(subject_embeddings)

In [220]:
from numpy import dot
from numpy.linalg import norm
import heapq
def detect_subject_from_query(query, subject_embeddings, top_k = 1):
    query_vec = embedder.embed([query])[0]

    sims = {
    code: dot(query_vec, emb) / (norm(query_vec) * norm(emb))
    for code, emb in subject_embeddings.items()
    } 

    top_subjects = heapq.nlargest(top_k, sims.items(), key=lambda x: x[1])
    return top_subjects
def detect_type_from_query(query, type_embeddings, top_k = 1):
    query_vec = embedder.embed([query])[0]

    sims = {
    code: dot(query_vec, emb) / (norm(query_vec) * norm(emb))
    for code, emb in type_embeddings.items()
    } 

    top_subjects = heapq.nlargest(top_k, sims.items(), key=lambda x: x[1])
    return top_subjects
def detect_term_from_query(query, term_embeddings, top_k = 1):
    query_vec = embedder.embed([query])[0]

    sims = {
    term: dot(query_vec, emb) / (norm(query_vec) * norm(emb))
    for term, emb in term_embeddings.items()
    } 

    top_terms = heapq.nlargest(top_k, sims.items(), key=lambda x: x[1])
    return top_terms

query_test_sub = "khoa học máy tính, deeplearning, ai"
main_sub = detect_subject_from_query(query_test_sub, subject_embeddings, top_k=5)
print(main_sub)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[('DPL302m', 0.7501127), ('ADY201m', 0.74621), ('AIG202c', 0.7443107), ('AIL303m', 0.734198), ('MAI391', 0.72010875)]


In [214]:
from deep_translator import GoogleTranslator

def translate_en_to_vi_google(text):
    return GoogleTranslator(source='en', target='vi').translate(text)
text = "welcome to the course on Search Engines. In this course, we will explore the fundamentals of search engine technology, including indexing, querying, and ranking algorithms. You will learn how to build efficient search systems and understand the challenges involved in processing large datasets."
translated_text = translate_en_to_vi_google(text)
print(f"Translated text: {translated_text}")

Translated text: Chào mừng bạn đến với khóa học về công cụ tìm kiếm. Trong khóa học này, chúng tôi sẽ khám phá các nguyên tắc cơ bản của công nghệ công cụ tìm kiếm, bao gồm lập chỉ mục, truy vấn và thuật toán xếp hạng. Bạn sẽ học cách xây dựng các hệ thống tìm kiếm hiệu quả và hiểu những thách thức liên quan đến việc xử lý các bộ dữ liệu lớn.


In [ ]:
type_descriptions = {
    "tổng kết môn học": "Tóm tắt và đánh giá tổng quan về một môn học, bao gồm điểm trung bình, trạng thái và điểm số tổng quan các loại điểm như lab, assignment, Participation, final exam.",
    "chi tiết điểm": "Thông tin chi tiết về điểm số của sinh viên trong các mục đánh giá khác nhau, bao gồm phân loại, trọng số và giá trị điểm chi tiết, ví dụ lab 1, lab 2, lab 3, progress test 1, 2, 3,...",
    "điểm danh": "Thông tin về việc điểm danh, hoặc thời gian biểu, lịch học sinh viên trong các buổi học, bao gồm ngày, ca học, phòng học, giảng viên và trạng thái điểm danh.",
    "thông tin sinh viên": "Thông tin cá nhân của sinh viên, bao gồm họ tên, ngày sinh, giới tính, địa chỉ, mã sinh viên và các thông tin liên quan đến học tập.",
}
type_embeddings = {
    type_name: embedder.embed([embedder.prefix + description])[0] for type_name, description in type_descriptions.items()}

query_test_type = "nghề nghiệp của bố"
main_type = detect_type_from_query(query_test_type, type_embeddings, top_k=5)
print(main_type)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[('thông tin sinh viên', 0.5223162), ('điểm danh', 0.5155834), ('chi tiết điểm', 0.500506), ('tổng kết môn học', 0.46564543)]


In [ ]:
term_descriptions = {
    "Fall2023": "Học kỳ mùa thu năm 2023, diễn ra từ tháng 9 đến tháng 12.",
    "Spring2024": "Học kỳ mùa xuân năm 2024, bắt đầu từ tháng 1 đến tháng 4.",
    "Summer2024": "Học kỳ hè năm 2024, diễn ra từ tháng 5 đến tháng 8.",
    "Fall2024": "Học kỳ mùa thu năm 2024, từ tháng 9 đến tháng 12.",
    "Spring2025": "Học kỳ mùa xuân năm 2025, từ tháng 1 đến tháng 4."
}

term_embeddings = {
    term_name: embedder.embed([embedder.prefix + description])[0]
    for term_name, description in term_descriptions.items()
}
query_test_term = "học kỳ tháng 9 "
main_term = detect_term_from_query(query_test_term, term_embeddings, top_k=5)
print(main_term)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[('Fall2024', 0.74627334), ('Fall2023', 0.7413152), ('Summer2024', 0.6566202), ('Spring2025', 0.6481493), ('Spring2024', 0.6343021)]


In [217]:
query = "lịch học CPV"
# Kiểm tra ngôn ngữ của query
from langdetect import detect
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False
if is_english(query):
    query = translate_en_to_vi_google(query)
    print(f"Translated query: {query}")
else:
    query = query.strip()
    print(f"Original query: {query}")

Original query: lịch học CPV


In [218]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

def search_qdrant_by_query(query, subject_embeddings, type_embeddings, client, collection_name="Fap_data_testing", limit=1):
    # Detect subjects and types
    detected_subjects = detect_subject_from_query(query, subject_embeddings)
    detected_types = detect_subject_from_query(query, type_embeddings)
    detected_terms = detect_term_from_query(query, term_embeddings)
    query_vec = embedder.embed([embedder.prefix + query])[0]

    # print(f"Detected subjects: {detected_subjects}")
    # print(f"Detected types: {detected_types}")

    must = []
    should = []

    if detected_types:
        print("🟩 Loại detect:", detected_types[0][0])
        should.append(FieldCondition(
            key="loai",
            match=MatchValue(value=detected_types[0][0])
        ))

    if detected_subjects:
        for subject_code, _ in detected_subjects:
            print("➡️ Mã môn học detect:", subject_code)
            should.append(FieldCondition(
                key="ma_mon_hoc",
                match=MatchValue(value=subject_code)
            ))
    
    if detected_terms:
        for term, _ in detected_terms:
            print("➡️ Học kỳ detect:", term)
            should.append(FieldCondition(
                key="hoc_ky",
                match=MatchValue(value=term)
            ))

    query_filter = Filter(must=must, should=should) if must or should else None

    hits = client.search(
        collection_name=collection_name,
        query_vector=query_vec.tolist(),
        limit=limit,
        query_filter=query_filter
    )

    for hit in hits:
        # print(f"\n🔍 Score: {hit.score:.4f}")
        # print(f"📘 Subject: {hit.payload.get('ma_mon_hoc')} - {hit.payload.get('loai')}")
        print(f"📄 Content:\n{hit.payload.get('noi_dung')}")
        print("--------")
    return hits

# Example usage:
query = "nghề nghiệp của bố"
hits = search_qdrant_by_query(query, subject_embeddings, type_embeddings, client)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🟩 Loại detect: thông tin sinh viên
➡️ Mã môn học detect: MAS291
➡️ Học kỳ detect: Fall2023


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23440\945788711.py:41: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


📄 Content:
LOẠI: Điểm danh
Sinh viên: DE190340
Môn học: MAS291 - Statistics & Probability
Học kỳ: Fall2024 | Buổi số: 8 - Ngày: Friday 27/09/2024 - Ca: 3_(12:30-14:45) - Phòng: Gamma 311.
Giảng viên: trangtt106 | Nhóm: AI18D02
Trạng thái: Present | Ghi chú: Không rõ
--------


In [219]:
test_queries = [
    "lịch học môn CPV301 tuần này",
    "điểm lab 1 môn DPL302m",
    "tóm tắt kết quả học tập học kỳ Fall2023",
    "giảng viên dạy môn Search Engines là ai",
    "điểm danh môn VOV114 ngày 12/10",
    "điểm trung bình môn AI18D01",
    "sinh viên DE190340 có học bổng không",
    "điểm final môn DBI202",
    "thông tin cá nhân sinh viên Phan Công Tâm",
    "kết quả bài kiểm tra giữa kỳ môn MAD101",
    "điểm tổng kết môn SWE201c",
    "điểm danh môn PFP191 tuần trước",
    "điểm lab 2 môn DPL302m",
    "điểm assignment môn DBI202",
    "điểm progress test 1 môn MAD101",
    "điểm giữa kỳ môn MAE101",
    "điểm cuối kỳ môn MAS291",
    "điểm danh môn SSL101c ngày 01/11",
    "giảng viên môn DSA103 là ai",
    "điểm bài tập lớn môn CSD203",
    "điểm danh môn VOV124 tuần này",
    "điểm lab 3 môn DWP301c",
    "điểm tổng kết môn SEG301",
    "điểm danh môn JPD113 ngày 15/10",
    "điểm final môn TMG301",
    "điểm assignment môn ITE303c",
    "điểm progress test 2 môn MAI391",
    "điểm danh môn JPD123 ngày 20/10",
    "điểm lab 1 môn SWE201c",
    "điểm tổng kết môn CEA201",
]
for i, q in enumerate(test_queries, 1):
    print(f"{i}. {q}")

print("\n--- Kết quả tìm kiếm cho từng truy vấn ---\n")
for stt, query in enumerate(test_queries, 1):
    print(f"\n🔎 Query {stt}: {query}")
    hits = search_qdrant_by_query(query, subject_embeddings, type_embeddings, client, limit=3)
    print(f"==> {len(hits)} kết quả\n")

1. lịch học môn CPV301 tuần này
2. điểm lab 1 môn DPL302m
3. tóm tắt kết quả học tập học kỳ Fall2023
4. giảng viên dạy môn Search Engines là ai
5. điểm danh môn VOV114 ngày 12/10
6. điểm trung bình môn AI18D01
7. sinh viên DE190340 có học bổng không
8. điểm final môn DBI202
9. thông tin cá nhân sinh viên Phan Công Tâm
10. kết quả bài kiểm tra giữa kỳ môn MAD101
11. điểm tổng kết môn SWE201c
12. điểm danh môn PFP191 tuần trước
13. điểm lab 2 môn DPL302m
14. điểm assignment môn DBI202
15. điểm progress test 1 môn MAD101
16. điểm giữa kỳ môn MAE101
17. điểm cuối kỳ môn MAS291
18. điểm danh môn SSL101c ngày 01/11
19. giảng viên môn DSA103 là ai
20. điểm bài tập lớn môn CSD203
21. điểm danh môn VOV124 tuần này
22. điểm lab 3 môn DWP301c
23. điểm tổng kết môn SEG301
24. điểm danh môn JPD113 ngày 15/10
25. điểm final môn TMG301
26. điểm assignment môn ITE303c
27. điểm progress test 2 môn MAI391
28. điểm danh môn JPD123 ngày 20/10
29. điểm lab 1 môn SWE201c
30. điểm tổng kết môn CEA201

--- Kế

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🟩 Loại detect: tổng kết môn học
➡️ Mã môn học detect: CEA201
➡️ Học kỳ detect: Spring2025
📄 Content:
LOẠI: tổng kết môn học
Môn học: CEA201 - Computer Organization and Architecture
Học kỳ: Summer2024
Điểm trung bình: 7.7
Trạng thái: Passed
Tóm tắt: {'Assignment': {'weight': '30.0 %', 'value': '6.8'}, 'Exercises': {'weight': '30.0 %', 'value': '8.3'}, 'Final exam': {'weight': '40.0 %', 'value': '8'}, 'Final exam Resit': {'weight': '40.0 %', 'value': None}}
--------
📄 Content:
LOẠI: Chi tiết điểm
Sinh viên: DE190340
Môn học: CEA201 - Computer Organization and Architecture
Học kỳ: Summer2024
Mục: Final exam Resit | Loại: Final exam Resit
Trọng số: 40.0 % | Điểm đạt: Khong ro
--------
📄 Content:
LOẠI: Chi tiết điểm
Sinh viên: DE190340
Môn học: CEA201 - Computer Organization and Architecture
Học kỳ: Summer2024
Mục: Final exam | Loại: Final exam
Trọng số: 40.0 % | Điểm đạt: 8.0
--------
==> 3 kết quả



In [3]:
# Load data file pdf hướng dẫn sinh viên dùng FAP 
import PyPDF2
import pandas as pd

# Đọc file PDF
pdf_path = r"D:\Desktop\tap tanh hoc code\.vscode\Summer_2025\SEG301\Fap_Chat\data\Hướng dẫn sử dụng và thực hiện các loại đơn từ.pdf"

try:
    with open(pdf_path, 'rb') as file:
        # Tạo PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)
        
        # Số trang của file PDF
        num_pages = len(pdf_reader.pages)
        print(f"Số trang trong file PDF: {num_pages}")
        
        # Đọc nội dung từng trang
        content = []
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            content.append(page.extract_text())
            print(page.extract_text())
            
        # In ra một số thống kê cơ bản
        total_chars = sum(len(text) for text in content)
        avg_chars_per_page = total_chars / num_pages
        
        print(f"\nTổng số ký tự: {total_chars}")
        print(f"Trung bình số ký tự mỗi trang: {avg_chars_per_page:.2f}")
        
        # In ra preview nội dung trang đầu tiên
        print("\nPreview nội dung trang đầu tiên:")
        print(content[0][:500] + "...")

except FileNotFoundError:
    print("Không tìm thấy file PDF tại đường dẫn đã cho")
except Exception as e:
    print(f"Có lỗi xảy ra khi đọc file PDF: {str(e)}")


Số trang trong file PDF: 34
   
 
0 
 
HƯỚNG DẪ N SỬ DỤNG FAP  VÀ 
THỰC HI ỆN CÁC LO ẠI ĐƠN T Ừ   
  

   
  
 
   Phòng Dịch vụ sinh viên FPTU Đà Nẵng  
Email: dvsv.fudn@fe.edu.vn  
Hotline:  (0236) 730 8686  
GIỚI THIỆU HỆ THỐNG FAP  
FAP  là viết tắt của FPT Academic Portal , là cổng thông tin trực tuyến của Đại học FPT.  
Hệ thống là kênh giao tiếp chính giữa nhà trường và sinh viên;  cung cấp nhiều tính năng hữu ích cho 
sinh viên, phụ huynh và giảng viên như:  
• Theo dõi  thông báo:  Tất cả t hông báo từ các bộ phận nhà trường: phòng Đào tạo, Khảo thí, 
Dịch vụ sinh viên, Công tác sinh viên, Hợp tác quốc tế và Phát triển cá nhân, Hành chính thư 
viện, Công nghệ thông tin, Kế toán,…  
• Tra cứu thông tin:  Lịch học, điểm thi, điểm danh, lịch thi, học phí,... 
• Thực hiện các loại đơn từ:  Đăng ký môn học, xin tạm ngừng, chuyển ngành, chuyển cơ sở , 
xác nhận sinh viên,...  
Vì vậy, sinh viên cần theo dõi các thông báo, điểm danh, lịch thi, lịch học, … thường xuyên trên FAP 
để ch